In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None) 

/Users/ghorafimanal/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### Traitement de la base départs

##### Importation de la base

Ici on doit expliquer pourquoi l'importation en ligne n'a pas fonctionner

In [2]:
df1=pd.read_excel("Detailed_Statistics_Departures.xlsx")
# Afficher les 5 premières lignes pour vérifier
df1.head()

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Wheels-off time,Taxi-Out time (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes)
0,AA,2020-01-01 00:00:00,1.0,N110AN,LAX,07:30:00,07:30:00,393.0,404.0,0.0,07:57:00,27.0,0.0,0.0,0.0,0.0,0.0
1,AA,2020-01-01 00:00:00,3.0,N111ZM,LAX,12:30:00,12:24:00,389.0,370.0,-6.0,12:38:00,14.0,0.0,0.0,0.0,0.0,0.0
2,AA,2020-01-01 00:00:00,111.0,N663AW,CLT,12:00:00,13:11:00,127.0,119.0,71.0,13:34:00,23.0,19.0,0.0,0.0,0.0,44.0
3,AA,2020-01-01 00:00:00,117.0,N113AN,LAX,19:30:00,19:26:00,402.0,379.0,-4.0,19:51:00,25.0,0.0,0.0,0.0,0.0,0.0
4,AA,2020-01-01 00:00:00,179.0,N103NN,SFO,10:30:00,10:25:00,409.0,392.0,-5.0,10:42:00,17.0,0.0,0.0,0.0,0.0,0.0


Les deux dernières lignes de notre fichier excel correspondent à la source des données. Ainsi, on peut les supprimer. 

In [3]:
# Identifier les index des deux dernières lignes
ind= df1.index[-2:]

# Supprimer ces lignes
df1 = df1.drop(ind)

##### Extraction du jour du vol de la colonne 'Date (MM/DD/YYYY)'


D'après la sortie du code précédent, nous pouvons clairement voir que la colonne Date (MM/DD/YYYY) ne contient pas uniquement la date du jour du vol mais aussi la chaîne de caractère 00:00:00 qui renvoie à l'heure du début d'un jour.

In [4]:
df1.rename(columns={'Date (MM/DD/YYYY)': 'Date'}, inplace=True)

In [5]:
df1['Date'] = pd.to_datetime(df1['Date'])

In [6]:
df1['Date'] = df1['Date'].dt.date

Nous souhaitons que la date soit au format "DD/MM/YYYY"

In [7]:
df1['Date'] =pd.to_datetime(df1['Date'], format="format='%m/%d/%Y")

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57410 entries, 0 to 57409
Data columns (total 17 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Carrier Code                              57410 non-null  object        
 1   Date                                      57410 non-null  datetime64[ns]
 2   Flight Number                             57410 non-null  float64       
 3   Tail Number                               57184 non-null  object        
 4   Destination Airport                       57410 non-null  object        
 5   Scheduled departure time                  57410 non-null  object        
 6   Actual departure time                     57410 non-null  object        
 7   Scheduled elapsed time (Minutes)          57410 non-null  float64       
 8   Actual elapsed time (Minutes)             57410 non-null  float64       
 9   Departure delay (Minutes)   

In [9]:
df1.head()

,Carrier Code,Date,Flight Number,Tail Number,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Wheels-off time,Taxi-Out time (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes)
0,AA,2020-01-01,1.0,N110AN,LAX,07:30:00,07:30:00,393.0,404.0,0.0,07:57:00,27.0,0.0,0.0,0.0,0.0,0.0
1,AA,2020-01-01,3.0,N111ZM,LAX,12:30:00,12:24:00,389.0,370.0,-6.0,12:38:00,14.0,0.0,0.0,0.0,0.0,0.0
2,AA,2020-01-01,111.0,N663AW,CLT,12:00:00,13:11:00,127.0,119.0,71.0,13:34:00,23.0,19.0,0.0,0.0,0.0,44.0
3,AA,2020-01-01,117.0,N113AN,LAX,19:30:00,19:26:00,402.0,379.0,-4.0,19:51:00,25.0,0.0,0.0,0.0,0.0,0.0
4,AA,2020-01-01,179.0,N103NN,SFO,10:30:00,10:25:00,409.0,392.0,-5.0,10:42:00,17.0,0.0,0.0,0.0,0.0,0.0


##### Selection des variables non porteuses de sens à notre problématique

In [10]:
#justification suppression variables vols : 
#Flight number : peu informatif pour la prédiction 
#Tail Number : permet d'identifier des pbs liés à des avions spécifiques (rare)
#Scheduled elapsed time / Actual elapsed time : le temps de vol prévu/réel pas pertinent pour prédire les retards

In [11]:
df1.drop(columns=["Carrier Code","Flight Number","Tail Number",
"Scheduled elapsed time (Minutes)","Actual elapsed time (Minutes)", "Departure delay (Minutes)"], inplace=True)

##### Création de variables pertinentes pour expliquer les retards

##### Création de la colonne Weekday_Flight

Nous aimerions aussi voir si le retard d'un vol peut être lié au jour de la semaine où le vol aura lieu. Pour cela, il est nécessaire de créer une colonne contenant le jour de la semaine (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday) correspondant à chaque vol de notre data frame.



In [12]:
# Convertir la colonne 'Date' en format datetime 
df1['Date'] = pd.to_datetime(df1['Date'])

# Créer la colonne Weekday_Flight avec les jours de la semaine correspondant aux dates des vols
df1['Weekday_Flight'] = df1['Date'].dt.day_name()

# Afficher Date et Weekday_Flight
print(df1[['Date','Weekday_Flight']])

            Date Weekday_Flight
0     2020-01-01      Wednesday
1     2020-01-01      Wednesday
2     2020-01-01      Wednesday
3     2020-01-01      Wednesday
4     2020-01-01      Wednesday
...          ...            ...
57405 2023-12-31         Sunday
57406 2023-12-31         Sunday
57407 2023-12-31         Sunday
57408 2023-12-31         Sunday
57409 2023-12-31         Sunday

[57410 rows x 2 columns]


##### Création de la colonne Season

La variable "saison" peut influencer les retards des vols en raison des variations météorologiques et des volumes de trafic spécifiques à chaque période de l'année. Par exemple, l’hiver apporte souvent des conditions difficiles comme la neige ou le brouillard, tandis que l’été, marqué par un trafic élevé, peut être perturbé par des orages ou des surcharges aéroportuaires. Ainsi, en tenant compte des saisons, il devient possible de mieux comprendre et anticiper les facteurs contribuant aux retards.



In [13]:
# Définir une fonction qui détermine pour une date données la saison correspondante
def get_season(date):
    year = date.year
    if date >= pd.Timestamp(year=year, month=3, day=21) and date < pd.Timestamp(year=year, month=6, day=21):
        return 'Printemps'
    elif date >= pd.Timestamp(year=year, month=6, day=21) and date < pd.Timestamp(year=year, month=9, day=23):
        return 'Été'
    elif date >= pd.Timestamp(year=year, month=9, day=23) and date < pd.Timestamp(year=year, month=12, day=21):
        return 'Automne'
    else:
        return 'Hiver'


# Convertir la colonne Date en datetime 
df1['Date'] = pd.to_datetime(df1['Date'])

# Appliquer la fonction sur la colonne Date du dataframe
df1['Season'] = df1['Date'].apply(get_season)

# Afficher  des observations aléatoires du DataFrame
print(df1.Season[1000:1010])
print(df1.Season[2000:2010])

1000    Printemps
1001    Printemps
1002    Printemps
1003    Printemps
1004    Printemps
1005    Printemps
1006    Printemps
1007    Printemps
1008    Printemps
1009    Printemps
Name: Season, dtype: object
2000    Automne
2001    Automne
2002    Automne
2003    Automne
2004    Automne
2005    Automne
2006    Automne
2007    Automne
2008    Automne
2009    Automne
Name: Season, dtype: object


##### Création de la colonne Period_Day

In [14]:
# Définir une fonction pour attribuer la période de la journée
def definir_periode(heure):
    if heure >= pd.to_datetime('06:00', format='%H:%M').time() and heure < pd.to_datetime('12:00', format='%H:%M').time():
        return 'Matin'
    elif heure >= pd.to_datetime('12:00', format='%H:%M').time() and heure < pd.to_datetime('18:00', format='%H:%M').time():
        return 'Après-midi'
    else:
        return 'Soir'

# Convertir la colonne en type datetime
df1['Scheduled departure time'] = pd.to_datetime(df1['Scheduled departure time'], format='%H:%M:%S')

# Appliquer la fonction pour créer une nouvelle colonne
df1['Period_Day'] = df1['Scheduled departure time'].dt.time.apply(definir_periode)

# Reconvertir les colonnes "Scheduled departure time" et "Actual departure time" pour ne garder que l'heure 
df1['Scheduled departure time'] = df1['Scheduled departure time'].dt.time

# Afficher les colonnes Period_Day et Scheduled departure time
print(df1[['Scheduled departure time','Period_Day']])

      Scheduled departure time  Period_Day
0                     07:30:00       Matin
1                     12:30:00  Après-midi
2                     12:00:00  Après-midi
3                     19:30:00        Soir
4                     10:30:00       Matin
...                        ...         ...
57405                 17:29:00  Après-midi
57406                 12:30:00  Après-midi
57407                 06:59:00       Matin
57408                 08:59:00       Matin
57409                 12:59:00  Après-midi

[57410 rows x 2 columns]


##### Création de notre variable cible Y=Delay

Le retard d'un vol au départ se définit comme l'écart entre l'heure de départ ou d'arrivée prévue d'un vol, telle qu'indiquée dans le programme, et l'heure réelle à laquelle le vol décolle.. Un vol est souvent considéré en retard si cet écart dépasse un seuil défini, par exemple 5 minutes.



In [15]:
# Convertir les colonnes "Scheduled departure time" et "Actual departure time" en objets datetime
df1['Scheduled departure time'] = pd.to_datetime(df1['Scheduled departure time'], format='%H:%M:%S')
df1['Actual departure time'] = pd.to_datetime(df1['Actual departure time'], format='%H:%M:%S')

# Création de la variable cible "Retard en calculant la différence en minutes et appliquer la condition
df1['Retard'] = ((df1['Actual departure time'] - df1['Scheduled departure time']).dt.total_seconds() / 60 > 0).astype(int)

# Afficher les colonnes "Actual departure time", "Scheduled departure time" et "Retard"
print(df1[['Actual departure time', 'Scheduled departure time', 'Retard']])

      Actual departure time Scheduled departure time  Retard
0       1900-01-01 07:30:00      1900-01-01 07:30:00       0
1       1900-01-01 12:24:00      1900-01-01 12:30:00       0
2       1900-01-01 13:11:00      1900-01-01 12:00:00       1
3       1900-01-01 19:26:00      1900-01-01 19:30:00       0
4       1900-01-01 10:25:00      1900-01-01 10:30:00       0
...                     ...                      ...     ...
57405   1900-01-01 17:25:00      1900-01-01 17:29:00       0
57406   1900-01-01 12:26:00      1900-01-01 12:30:00       0
57407   1900-01-01 06:57:00      1900-01-01 06:59:00       0
57408   1900-01-01 08:53:00      1900-01-01 08:59:00       0
57409   1900-01-01 12:57:00      1900-01-01 12:59:00       0

[57410 rows x 3 columns]


In [16]:

# Reconvertir les colonnes "Scheduled departure time" et "Actual departure time" pour ne garder que l'heure 
df1['Scheduled departure time'] = df1['Scheduled departure time'].dt.time
df1['Actual departure time'] = df1['Actual departure time'].dt.time

# Afficher les colonnes "Actual departure time", "Scheduled departure time" et "Retard"
print(df1[['Actual departure time', 'Scheduled departure time', 'Retard']])

      Actual departure time Scheduled departure time  Retard
0                  07:30:00                 07:30:00       0
1                  12:24:00                 12:30:00       0
2                  13:11:00                 12:00:00       1
3                  19:26:00                 19:30:00       0
4                  10:25:00                 10:30:00       0
...                     ...                      ...     ...
57405              17:25:00                 17:29:00       0
57406              12:26:00                 12:30:00       0
57407              06:57:00                 06:59:00       0
57408              08:53:00                 08:59:00       0
57409              12:57:00                 12:59:00       0

[57410 rows x 3 columns]


##### Traitement des variables catégorielle

In [17]:
#on initialise l'encodeur
label_encoder = LabelEncoder()

##### Encodage de la variable Weekday_Flight

In [18]:
#On affiche les différentes modalités de la variable
df1['Weekday_Flight'].unique()

array(['Wednesday', 'Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday',
       'Thursday'], dtype=object)

In [19]:
df1['Weekday_Flight'] = label_encoder.fit_transform(df1['Weekday_Flight'])

##### Encodage de la variable Season 

In [20]:
df1['Season'].unique()

array(['Hiver', 'Printemps', 'Été', 'Automne'], dtype=object)

In [21]:
df1['Season'] = label_encoder.fit_transform(df1['Season'])

##### Encodage de la variable Period_Day

In [22]:
df1['Period_Day'].unique()

array(['Matin', 'Après-midi', 'Soir'], dtype=object)

In [23]:
df1['Period_Day'] = label_encoder.fit_transform(df1['Period_Day'])

##### Encodage de la variable Destination Airport

In [24]:
df1['Destination Airport'].unique()

array(['LAX', 'CLT', 'SFO', 'MIA', 'ORD', 'BOS', 'EGE', 'PHX', 'DFW',
       'AUS', 'DCA', 'SNA', 'JAC', 'STT', 'LAS', 'RDU', 'FLL', 'SAT',
       'SAN'], dtype=object)

In [25]:
df1['Destination Airport'] = label_encoder.fit_transform(df1['Destination Airport'])

In [26]:
df1.head()

,Date,Destination Airport,Scheduled departure time,Actual departure time,Wheels-off time,Taxi-Out time (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes),Weekday_Flight,Season,Period_Day,Retard
0,2020-01-01,9,07:30:00,07:30:00,07:57:00,27.0,0.0,0.0,0.0,0.0,0.0,6,1,1,0
1,2020-01-01,9,12:30:00,12:24:00,12:38:00,14.0,0.0,0.0,0.0,0.0,0.0,6,1,0,0
2,2020-01-01,2,12:00:00,13:11:00,13:34:00,23.0,19.0,0.0,0.0,0.0,44.0,6,1,0,1
3,2020-01-01,9,19:30:00,19:26:00,19:51:00,25.0,0.0,0.0,0.0,0.0,0.0,6,1,2,0
4,2020-01-01,16,10:30:00,10:25:00,10:42:00,17.0,0.0,0.0,0.0,0.0,0.0,6,1,1,0


##### Traitement des valeurs manquantes

In [27]:
# Percentage of missing values

miss_values= df1.isnull().sum().sort_values(ascending=False)  
miss_values

percentage= (df1.isnull().sum()/df1.isnull().count().sort_values(ascending=False))
missing_data= pd.concat([miss_values, percentage], axis=1, keys=['Number of missing values', 'Percent (%)'])

missing_data

,Number of missing values,Percent (%)
Date,0,0.0
Destination Airport,0,0.0
Scheduled departure time,0,0.0
Actual departure time,0,0.0
Wheels-off time,0,0.0
Taxi-Out time (Minutes),0,0.0
Delay Carrier (Minutes),0,0.0
Delay Weather (Minutes),0,0.0
Delay National Aviation System (Minutes),0,0.0
Delay Security (Minutes),0,0.0


Il n'y a pas de valeurs manquantes

### Traitement de la base météorologique

In [28]:
df2=pd.read_excel("données_météo.xlsx")

In [29]:
df2['time']=pd.to_datetime(df2['time'], format="format='%m/%d/%Y")

In [30]:
df2.head(5)

,time,temperature_2m (°C),relative_humidity_2m (%),dew_point_2m (°C),apparent_temperature (°C),precipitation (mm),rain (mm),snowfall (cm),snow_depth (m),weather_code (wmo code),pressure_msl (hPa),surface_pressure (hPa),cloud_cover (%),cloud_cover_low (%),cloud_cover_mid (%),cloud_cover_high (%),et0_fao_evapotranspiration (mm),vapour_pressure_deficit (kPa),wind_speed_10m (km/h),wind_speed_100m (km/h),wind_direction_10m (°),wind_direction_100m (°),wind_gusts_10m (km/h),soil_temperature_0_to_7cm (°C),soil_temperature_7_to_28cm (°C),soil_temperature_28_to_100cm (°C),soil_temperature_100_to_255cm (°C),soil_moisture_0_to_7cm (m³/m³),soil_moisture_7_to_28cm (m³/m³),soil_moisture_28_to_100cm (m³/m³),soil_moisture_100_to_255cm (m³/m³)
0,2020-11-01 00:00:00,7.4,64,1.1,4.1,0.0,0.0,0.0,0.0,3,1031.7,1031.4,100,94,0,0,0.02,0.37,9.1,14.7,189,191,19.8,8.0,9.9,15.6,19.7,0.385,0.392,0.276,0.327
1,2020-11-01 01:00:00,7.3,67,1.6,4.2,0.0,0.0,0.0,0.0,3,1031.2,1030.9,96,96,0,0,0.02,0.34,8.0,15.9,190,193,15.1,8.0,9.9,15.5,19.7,0.384,0.391,0.276,0.327
2,2020-11-01 02:00:00,7.9,67,2.1,4.7,0.0,0.0,0.0,0.0,3,1030.9,1030.6,89,89,0,0,0.02,0.35,9.4,17.9,187,189,15.5,8.0,9.8,15.5,19.7,0.384,0.390,0.276,0.327
3,2020-11-01 03:00:00,8.7,64,2.2,4.7,0.0,0.0,0.0,0.0,2,1030.0,1029.8,76,76,0,0,0.04,0.41,14.8,23.9,186,187,24.1,8.2,9.8,15.5,19.7,0.383,0.390,0.276,0.327
4,2020-11-01 04:00:00,9.0,64,2.4,4.9,0.0,0.0,0.0,0.0,3,1029.3,1029.1,95,95,0,0,0.05,0.42,16.1,25.6,209,208,28.4,8.4,9.8,15.4,19.7,0.383,0.389,0.277,0.327


#### Selection de nos variables

In [31]:
#justification suppression variables météo : 
#apparent_temperature deja une combi de temperature_2m et relative_humidity_2m
#precipitation = pluie + neige + averses
#wind_speed_100m
#shortwave_radiation, diffuse_radiation : liées au rayonnement solaire
#et0_fao_evapotranspiration
#soil_temperature_*, soil_moisture_*)

La variable cloud_cover représente la couverture nuageuse totale, c'est-à-dire la fraction de l'horizon couvert par les nuages. C'est une variable pertinente car elle affecte directement la visibilité et la sécurité des vols (notamment pour les décollages et les atterissages). 
La variable cloud_cover_low représente la couverture nuageuse basse (en dessous de 2km). Cela peut etre associé à la formation de brouillard ou de nuages bas. 

La variable wind_speed_10m représente la vitesse du vent à 10 mètres. C'est une variable pertinente car le vent peut affecter la sécurité des opérations aériennes et provoquer des retards. 
La vitesse du vent à 10 mètres est le niveau standard.
La variable wind_direction_10m représente la direction du vent à 10 mètres. La direction du vent peut affecter les décollages et atterrissages car les avions ont besoin d'un vent favorable (vent de face) pour effectuer un décollage ou un atterissage en toute sécurité. 
La variable wind_gusts_10m représente les rafales de vent. 

In [32]:
df2.drop(columns=["soil_moisture_0_to_7cm (m³/m³)", "soil_moisture_7_to_28cm (m³/m³)", 
"soil_moisture_28_to_100cm (m³/m³)", "soil_moisture_100_to_255cm (m³/m³)",
"soil_temperature_28_to_100cm (°C)", "soil_temperature_100_to_255cm (°C)",
"soil_temperature_7_to_28cm (°C)", "soil_temperature_0_to_7cm (°C)", 
"cloud_cover_mid (%)", "cloud_cover_high (%)", "wind_speed_100m (km/h)",
"wind_direction_100m (°)", "pressure_msl (hPa)", "et0_fao_evapotranspiration (mm)"
, "apparent_temperature (°C)", "rain (mm)", "snowfall (cm)" ], inplace=True)

#### Traitement des valeurs manquantes

In [33]:
# Percentage of missing values

miss_values= df2.isnull().sum().sort_values(ascending=False)  
miss_values

percentage= (df2.isnull().sum()/df2.isnull().count().sort_values(ascending=False))
missing_data= pd.concat([miss_values, percentage], axis=1, keys=['Number of missing values', 'Percent (%)'])

missing_data

,Number of missing values,Percent (%)
snow_depth (m),480,0.01368
time,0,0.00000
temperature_2m (°C),0,0.00000
relative_humidity_2m (%),0,0.00000
dew_point_2m (°C),0,0.00000
precipitation (mm),0,0.00000
weather_code (wmo code),0,0.00000
surface_pressure (hPa),0,0.00000
cloud_cover (%),0,0.00000
cloud_cover_low (%),0,0.00000


In [34]:
#on remplace par la moyenne 
df2['snow_depth (m)'] = df2['snow_depth (m)'].fillna(df2['snow_depth (m)'].mean())

### Fusion des deux bases de données

In [35]:
df1['time'] = pd.to_datetime(df1['Date'].astype(str) + ' ' + df1['Scheduled departure time'].astype(str))

In [36]:
df1['time'] = df1['time'].dt.round('H')

/var/folders/z4/l1g2ltz55419x6y847ly0r800000gn/T/ipykernel_11522/786802130.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df1['time'] = df1['time'].dt.round('H')


In [37]:
df1.head(10)

,Date,Destination Airport,Scheduled departure time,Actual departure time,Wheels-off time,Taxi-Out time (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes),Weekday_Flight,Season,Period_Day,Retard,time
0,2020-01-01,9,07:30:00,07:30:00,07:57:00,27.0,0.0,0.0,0.0,0.0,0.0,6,1,1,0,2020-01-01 08:00:00
1,2020-01-01,9,12:30:00,12:24:00,12:38:00,14.0,0.0,0.0,0.0,0.0,0.0,6,1,0,0,2020-01-01 12:00:00
2,2020-01-01,2,12:00:00,13:11:00,13:34:00,23.0,19.0,0.0,0.0,0.0,44.0,6,1,0,1,2020-01-01 12:00:00
3,2020-01-01,9,19:30:00,19:26:00,19:51:00,25.0,0.0,0.0,0.0,0.0,0.0,6,1,2,0,2020-01-01 20:00:00
4,2020-01-01,16,10:30:00,10:25:00,10:42:00,17.0,0.0,0.0,0.0,0.0,0.0,6,1,1,0,2020-01-01 10:00:00
5,2020-01-01,10,14:45:00,14:45:00,15:01:00,16.0,0.0,0.0,0.0,0.0,0.0,6,1,0,0,2020-01-01 15:00:00
6,2020-01-01,9,09:00:00,08:58:00,09:17:00,19.0,0.0,0.0,0.0,0.0,0.0,6,1,1,0,2020-01-01 09:00:00
7,2020-01-01,9,20:30:00,20:26:00,20:58:00,32.0,0.0,0.0,0.0,0.0,0.0,6,1,2,0,2020-01-01 20:00:00
8,2020-01-01,9,18:30:00,18:24:00,18:50:00,26.0,0.0,0.0,0.0,0.0,0.0,6,1,2,0,2020-01-01 18:00:00
9,2020-01-01,9,17:00:00,16:58:00,17:15:00,17.0,0.0,0.0,0.0,0.0,0.0,6,1,0,0,2020-01-01 17:00:00


In [38]:
df=pd.merge(df1, df2, on='time', how='inner')

In [39]:
df.head(10)

,Date,Destination Airport,Scheduled departure time,Actual departure time,Wheels-off time,Taxi-Out time (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes),Weekday_Flight,Season,Period_Day,Retard,time,temperature_2m (°C),relative_humidity_2m (%),dew_point_2m (°C),precipitation (mm),snow_depth (m),weather_code (wmo code),surface_pressure (hPa),cloud_cover (%),cloud_cover_low (%),vapour_pressure_deficit (kPa),wind_speed_10m (km/h),wind_direction_10m (°),wind_gusts_10m (km/h)
0,2021-01-01,9,09:00:00,08:55:00,09:15:00,20.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0,2021-01-01 09:00:00,-1.8,83,-4.3,0.0,0.0,3,1031.7,99,0,0.09,9.0,2,16.2
1,2021-01-01,9,18:05:00,18:04:00,18:32:00,28.0,0.0,0.0,0.0,0.0,0.0,0,1,2,0,2021-01-01 18:00:00,4.0,65,-1.9,0.0,0.0,3,1030.2,100,0,0.28,10.6,100,22.7
2,2021-01-01,16,07:30:00,07:25:00,07:40:00,15.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0,2021-01-01 08:00:00,-1.6,82,-4.3,0.0,0.0,0,1031.8,14,0,0.10,9.2,349,16.2
3,2021-01-01,2,11:52:00,11:46:00,12:02:00,16.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0,2021-01-01 12:00:00,-1.2,81,-4.0,0.0,0.0,3,1032.9,100,0,0.10,10.0,21,19.1
4,2021-01-01,10,14:30:00,14:24:00,14:40:00,16.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,2021-01-01 14:00:00,0.8,73,-3.5,0.0,0.0,3,1033.5,99,0,0.17,11.6,36,20.9
5,2021-01-01,12,09:30:00,09:24:00,09:43:00,19.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0,2021-01-01 10:00:00,-1.7,83,-4.2,0.0,0.0,3,1032.1,91,0,0.09,9.8,6,15.8
6,2021-01-01,12,13:53:00,13:48:00,14:03:00,15.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,2021-01-01 14:00:00,0.8,73,-3.5,0.0,0.0,3,1033.5,99,0,0.17,11.6,36,20.9
7,2021-01-01,5,10:20:00,10:12:00,10:28:00,16.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0,2021-01-01 10:00:00,-1.7,83,-4.2,0.0,0.0,3,1032.1,91,0,0.09,9.8,6,15.8
8,2021-01-01,10,16:18:00,16:12:00,16:30:00,18.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,2021-01-01 16:00:00,3.1,64,-3.0,0.0,0.0,3,1033.4,100,0,0.27,6.7,106,17.6
9,2021-01-01,10,06:00:00,05:59:00,06:27:00,28.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0,2021-01-01 06:00:00,-0.8,77,-4.4,0.0,0.0,0,1030.4,0,0,0.14,10.4,358,17.6


##### Vérification de la multicolinéarité

Nous devons vérifier la corrélation entre nos variables explicatives.

In [45]:
X = df.drop(columns=['Retard','Scheduled departure time', "Actual departure time", 'Wheels-off time', 'time', 'Date'])

In [46]:
correlation_matrix = X.corr()
print(correlation_matrix)

                                          Destination Airport  \
Destination Airport                                  1.000000   
Taxi-Out time (Minutes)                              0.054998   
Delay Carrier (Minutes)                             -0.007384   
Delay Weather (Minutes)                             -0.006136   
Delay National Aviation System (Minutes)             0.008374   
Delay Security (Minutes)                            -0.002433   
Delay Late Aircraft Arrival (Minutes)               -0.014928   
Weekday_Flight                                       0.000856   
Season                                               0.008984   
Period_Day                                          -0.025974   
temperature_2m (°C)                                 -0.000435   
relative_humidity_2m (%)                             0.020278   
dew_point_2m (°C)                                    0.007258   
precipitation (mm)                                  -0.000780   
snow_depth (m)           

On remarque que la corrélation entre les variables wind_speed_10m (qui représente la vitesse du vent à 10m) et wind_gusts_10m (qui représente les rafales de vent) est de 0,95. Ainsi, nous choissons de supprimer la variable wind_gusts_10m.

La corrélation entre les variables temperature_2m (qui représente la température de l'air à 2 mètres au-dessus du sol) et dew_point_2m (qui représente la température du point de rosée) est égalemement elevé (0,91). Nous supprimerons la variable dew_point_2m. 

In [48]:
df.drop(columns=['wind_gusts_10m (km/h)', 'dew_point_2m (°C)'], inplace=True)

In [34]:
df.to_csv('data.csv', index=False)